<a href="https://colab.research.google.com/github/omidbazgirTTU/LLMs/blob/main/RLHF_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RLHF Pipeline

Author: Ehsan Kamalinejad (EK)
Created: 2023-08-23

### References
- RLHF from the scratch [series](https://www.youtube.com/@ehsankamalinejad/playlists)
- The original [instructGPT paper](https://arxiv.org/pdf/2203.02155.pdf)
- The generalized advantage estimation [GAE paper](https://arxiv.org/abs/1506.02438)
- The [Stable Baseline RL repo](https://stable-baselines.readthedocs.io/en/master/modules/ppo2.html)

In [1]:
%%writefile requirements.txt
transformers
datasets
accelerate
scipy
omegaconf
torchtyping
matplotlib
ipywidgets

Writing requirements.txt


In [2]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=c6977f20644d54e8e295af06fabfe9f456f7056295f5973a04819138edaf6f9d
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully bu

In [3]:
import random
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pylab as plt
from omegaconf import DictConfig
from dataclasses import dataclass
from torchtyping import TensorType
from typing import Iterable, Sequence, List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from datasets import load_dataset

from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
)

In [4]:
set_seed(2023)

args = {
    'generator_model_name_or_path': 'EKKam/opt-1.5b_imdb_sft',
    'reward_model_name_or_path': 'EKKam/opt350m_imdb_sentiment_reward',
    'seq_length': 1024,
    'batch_size': 64,
    'lr': 0.00006,
    'prompt_size': 30,
    'prompt_batch_size': 128,
    'num_rollouts': 128,
    'epochs': 100,
    'ppo_epochs': 4,
    'gen_kwargs': {
        'max_new_tokens': 40,
        'top_k': 0,
        'top_p': 1.0,
        'do_sample': True
    },
    'kl_coef': 0.01,
    'gamma': 1,
    'lam': 0.95,
    'cliprange': 0.2,
    'cliprange_value': 0.2,
    'vf_coef': 1,
}

args = DictConfig(args)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(args.generator_model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

## Data Preparation

In [6]:
class PromptDataset():
    def __init__(self, prompt_size, data):
        all_input_ids = tokenizer([item for item in data['train']['text']]).input_ids
        self.prompts_input_ids = np.array([item[:prompt_size] for item in all_input_ids if len(item)>prompt_size])

    def __getitem__(self, ix):
        return self.prompts_input_ids[ix]

    def __len__(self):
        return len(self.prompts_input_ids)

In [7]:
class CustomPromptDataGenerator():
    def __init__(self, prompt_dataset, prompt_batch_size):
        self.prompt_dataset = prompt_dataset
        self.prompt_batch_size = prompt_batch_size

    def __iter__(self):
        self.dataset_indices = np.arange(len(self.prompt_dataset))
        return self

    def __next__(self):
        if len(self.dataset_indices) >= self.prompt_batch_size:
            picked_indices = np.random.choice(np.arange(len(self.dataset_indices)),
                                              self.prompt_batch_size,
                                              replace=False)
            samples = self.prompt_dataset[self.dataset_indices[picked_indices]]
            self.dataset_indices = np.delete(self.dataset_indices, picked_indices)
            input_ids = torch.tensor(samples)
            attention_mask = torch.ones_like(input_ids)
            batch = {'input_ids': input_ids, 'attention_mask': attention_mask}
            return batch
        else:
            raise StopIteration

## Rollout Storage Definition

In [8]:
@dataclass
class PPORLElement:
    query_tensor: TensorType["query_size"]
    response_tensor: TensorType["response_size"]
    logprobs: TensorType["response_size", "vocab_size"]
    values: TensorType["response_size"]
    rewards: TensorType["response_size"]


@dataclass
class PPORLBatch:
    query_tensors: TensorType["batch_size", "query_size"]
    response_tensors: TensorType["batch_size", "response_size"]
    logprobs: TensorType["batch_size", "response_size", "vocab_size"]
    values: TensorType["batch_size", "response_size"]
    rewards: TensorType["batch_size", "response_size"]


class PPORolloutStorage():

    def __init__(self):
        super().__init__()
        self.pad_token_id = tokenizer.pad_token_id
        self.history: Iterable[PPORLElement] = [None]

    def push(self, exps: Iterable[PPORLElement]):
        self.history += exps

    def clear_history(self):
        self.history = []

    def __getitem__(self, index: int) -> PPORLElement:
        return self.history[index]

    def __len__(self) -> int:
        return len(self.history)

    def create_loader(self, mini_batch_size: int, shuffle: bool) -> DataLoader:
        def collate_fn(elems: Iterable[PPORLElement]):
            return PPORLBatch(
                pad_sequence(
                    [elem.query_tensor for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ),
                pad_sequence(
                    [elem.response_tensor for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ),
                pad_sequence(
                    [elem.logprobs for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ),
                pad_sequence(
                    [elem.values for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True
                ),
                pad_sequence(
                    [elem.rewards for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ),
            )

        return DataLoader(self, mini_batch_size, shuffle=shuffle, collate_fn=collate_fn)

## LLM as an RL Agent

In [9]:
class Agent(nn.Module):
    def __init__(self, trainable=False):
        super().__init__()
        self.trainable = trainable
        self.model = AutoModelForCausalLM.from_pretrained(
            args.generator_model_name_or_path,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        if not self.trainable:
            self.model = self.model.eval()
            self.model.requires_grad_(False)
        else:
            n_embd = self.model.lm_head.in_features
            num_labels = 1
            self.value_head = nn.Sequential(
                nn.LayerNorm(n_embd),
                nn.GELU(),
                nn.Linear(n_embd, 4*n_embd),
                nn.GELU(),
                nn.Linear(4*n_embd, num_labels),
            ).to(torch.bfloat16).to(self.model.device)
        self.logit_head = self.model.get_output_embeddings()

    def generate(self, input_ids, **x):
        return self.model.generate(input_ids, **x)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids,
                             attention_mask=attention_mask,
                             output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]
        lm_logits = self.logit_head(last_hidden_state)
        if self.trainable:
            value = self.value_head(last_hidden_state).squeeze(-1)
            return lm_logits, value
        else:
            return lm_logits

## Reward Model

In [10]:
reward_model = AutoModelForSequenceClassification.from_pretrained(
    args.reward_model_name_or_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    num_labels=1,
)

reward_model = reward_model.eval()

config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

In [ ]:
def reward_fn(samples):
    ins = tokenizer(samples, padding=True, truncation=True, max_length=args.seq_length, return_tensors='pt')
    logits = reward_model(**ins.to(reward_model.device)).logits
    temperature = 0.3
    sentiments = torch.sigmoid(logits*temperature)[:,0].detach().cpu().tolist()
    return sentiments

## Rolout Creation

In [11]:
def logprobs_from_logits(logits, labels):
    logprobs = F.log_softmax(logits, dim=-1)
    logprobs_labels = torch.gather(logprobs, dim=-1, index=labels.unsqueeze(-1))
    return logprobs_labels.squeeze(-1)

In [13]:
class RolloutCreator():

    def __init__(
            self,
            prompt_dataset,
            prompt_batch_size=args.prompt_batch_size,
    ):
        self.prompt_batch_size = prompt_batch_size
        self.prompt_dataset = prompt_dataset
        self.prompt_generator = CustomPromptDataGenerator(self.prompt_dataset, self.prompt_batch_size)
        self.prompt_iterator = iter(self.prompt_generator)
        self.generate_kwargs = dict(
            args.gen_kwargs,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )


    def make_experience(self, model, num_rollouts=128):

        all_rollouts = []
        while len(all_rollouts) < num_rollouts:
            try:
                batch = next(self.prompt_iterator)
            except StopIteration:
                self.prompt_generator = CustomPromptDataGenerator(self.prompt_dataset, self.prompt_batch_size)
                self.prompt_iterator = iter(self.prompt_generator)
                batch = next(self.prompt_iterator)

            query_tensors = batch['input_ids'].to(model.model.device)
            trajectories = model.generate(
                query_tensors,
                attention_mask=batch['attention_mask'].to(model.model.device),
                **self.generate_kwargs
            )
            response_tensors = trajectories[:, query_tensors.shape[1]:]
            attention_mask = trajectories.not_equal(tokenizer.pad_token_id).long()

            with torch.no_grad():
                logits, values = model(
                    trajectories,
                    attention_mask=attention_mask,
                )
                ref_logits = ref_model(
                    trajectories,
                    attention_mask=attention_mask,
                )
            logprobs = logprobs_from_logits(logits[:, :-1, :], trajectories[:, 1:])
            ref_logprobs = logprobs_from_logits(ref_logits[:, :-1, :], trajectories[:, 1:])
            n_trajectories = trajectories.shape[0]
            values = values[:, :-1]

            start = batch['input_ids'].shape[1] - 1
            ends = start + attention_mask[:, start:].sum(1)
            truncated_values = [values[i, start : ends[i]] for i in range(n_trajectories)]
            truncated_logprobs = [logprobs[i, start : ends[i]] for i in range(n_trajectories)]

            texts = tokenizer.batch_decode(trajectories, skip_special_tokens=True)
            scores = reward_fn(texts)
            rewards = -args.kl_coef * (logprobs - ref_logprobs)
            all_rewards = [None] * n_trajectories
            for i in range(n_trajectories):
                rs = rewards[i][start : ends[i]]
                rs[-1] = scores[i]
                all_rewards[i] = rs

            new_rollout = [
                PPORLElement(
                    query_tensor=query_tensors[i],
                    response_tensor=response_tensors[i],
                    logprobs=truncated_logprobs[i],
                    values=truncated_values[i],
                    rewards=all_rewards[i],
                )
                for i in range(n_trajectories)
            ]
            all_rollouts += new_rollout

        score = torch.tensor(scores).mean().detach().cpu().item()

        return all_rollouts, score

## Losses

In [14]:
def gae(
    values,
    rewards,
):
    advantages = torch.zeros_like(rewards, device=rewards.device)
    last_advantage = 0
    last_value = 0
    with torch.no_grad():
        for t in reversed(range(rewards.shape[1])):
            delta = rewards[:, t] + args.gamma * last_value - values[:, t]
            last_advantage = delta + args.gamma * args.lam * last_advantage
            advantages[:, t] = last_advantage
            last_value = values[:, t]
        returns = advantages + values
    return advantages, returns

In [15]:
def ppo_loss(
    logprobs,
    values,
    old_logprobs,
    old_values,
    advantages,
    returns,
    mask,
):
    values_clipped = torch.clamp(
        values,
        old_values - args.cliprange_value,
        old_values + args.cliprange_value,
    )
    n = mask.sum()
    vf_loss1 = (values - returns) ** 2
    vf_loss2 = (values_clipped - returns) ** 2
    vf_loss = 0.5 * torch.sum(torch.max(vf_loss1, vf_loss2) * mask) / n
    log_ratio = (logprobs - old_logprobs) * mask
    ratio = torch.exp(log_ratio)
    pg_loss1 = -advantages * ratio
    pg_loss2 = -advantages * torch.clamp(ratio, 1.0 - args.cliprange, 1.0 + args.cliprange)
    pg_loss = torch.sum(torch.max(pg_loss1, pg_loss2) * mask) / n
    pg_clipfrac = torch.sum((pg_loss2 > pg_loss1).float() * mask) / n
    loss = pg_loss + args.vf_coef * vf_loss
    return loss

In [16]:
def loss_fn(mini_batch):
    query_tensors = mini_batch.query_tensors
    response_tensors = mini_batch.response_tensors
    old_logprobs = mini_batch.logprobs
    old_values = mini_batch.values
    old_rewards = mini_batch.rewards

    response_length = old_rewards.shape[1]

    advantages, returns = gae(old_values, old_rewards)

    trajectories = torch.hstack([mini_batch.query_tensors, mini_batch.response_tensors])
    attention_mask = trajectories.not_equal(tokenizer.pad_token_id).long()
    logits, values_pred = model(trajectories, attention_mask=attention_mask)

    values_pred = values_pred[:, :-1]
    logprobs = logprobs_from_logits(logits[:, :-1, :], trajectories[:, 1:])
    attention_mask = attention_mask[:, :-1]

    start = query_tensors.shape[1] - 1
    end = start + response_length
    logprobs, values_pred, mask = (
        logprobs[:, start:end],
        values_pred[:, start:end],
        attention_mask[:, start:end],
    )

    loss = ppo_loss(
        logprobs=logprobs,
        values=values_pred,
        old_logprobs=old_logprobs,
        old_values=old_values,
        advantages=advantages,
        returns=returns,
        mask=mask,
    )

    return loss, old_rewards[:,-1].mean().item()

## Inference before alignment

In [ ]:
ref_model = Agent(trainable=False)
model = Agent(trainable=True)

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
ins = tokenizer(
    ["This is an action Western.", "I saw this movie recently because"],
    return_tensors='pt',
    padding=True)

with torch.no_grad():
    outs = model.generate(
        ins['input_ids'].to(ref_model.model.device),
        attention_mask=ins['attention_mask'].to(ref_model.model.device),
        max_new_tokens=128,
        do_sample=True,
        temperature=0.5,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=2.,
    )

for i in range(len(outs)):
    generated_text = tokenizer.decode(outs[i], skip_special_tokens=True)
    print("\n" + "\033[1;30m" + generated_text)
    print("\033[1;32m" +'Score: ', np.round(reward_fn([generated_text]), decimals=4)[0], "\n")

## Training loop

In [ ]:
imdb = load_dataset("imdb")
prompt_dataset = PromptDataset(args.prompt_size, imdb)

store = PPORolloutStorage()
rollout_creator = RolloutCreator(prompt_dataset, prompt_batch_size=args.prompt_batch_size)

opt = torch.optim.AdamW(model.parameters(), args.lr)

In [ ]:
total_steps = (args.num_rollouts//args.batch_size)*args.ppo_epochs*args.epochs
tbar = tqdm(initial=0, total=total_steps)
all_scores = []

for i in range(args.epochs):

    # filling in the storage (phase 1)
    store.clear_history()
    rollouts, score = rollout_creator.make_experience(model, args.num_rollouts)
    store.push(rollouts)
    train_dataloader = store.create_loader(args.batch_size, shuffle=True)

    # loss calculation and graident optimization (Phase 2)
    for batch in train_dataloader:
        for _ in range(args.ppo_epochs):
            loss, reward = loss_fn(batch)
            loss.backward()
            opt.step()
            opt.zero_grad()
            tbar.update()
    all_scores.append(score)
    tbar.set_description(f"| score: {score:.3f} |")

In [ ]:
plt.plot(all_scores);

## Inference after alignment

In [ ]:
ins = tokenizer(
    ["This is an action Western.", "I saw this movie recently because"],
    return_tensors='pt',
    padding=True
)

with torch.no_grad():
    outs = model.generate(
        ins['input_ids'].to(ref_model.model.device),
        attention_mask=ins['attention_mask'].to(ref_model.model.device),
        max_new_tokens=128,
        do_sample=True,
        temperature=0.5,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=2.,
    )

for i in range(len(outs)):
    generated_text = tokenizer.decode(outs[i], skip_special_tokens=True)
    print("\n" + "\033[1;30m" + generated_text)
    print("\033[1;32m" +'Score: ', np.round(reward_fn([generated_text]), decimals=4)[0], "\n")